In [1]:
# %pip install pandas 
# %pip install tensorflow
# %pip install scikit-learn

In [2]:
%pip install pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data
n_samples = 1000
n_features = 286

# Features with some correlation to the target
X = np.random.randn(n_samples, n_features)
# Introduce NaNs in some features (random 10% missing values)


# Generate target variable with correlation to the first few features
# Adding noise for realistic correlation
y = 5 * X[:, 0] + 2 * X[:, 1] - 3 * X[:, 2] + np.random.randn(n_samples) * 0.5
nan_mask = np.random.rand(n_samples, n_features) < 0.1
X[nan_mask] = np.nan
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert X_train and X_test to Pandas DataFrames to simulate your setup
train_x = pd.DataFrame(X_train)
test_x = pd.DataFrame(X_test)

train_y = y_train
test_y = y_test

# Verify the number of NaN values
print("Number of NaNs in training data:", np.isnan(train_x).sum().sum())

Number of NaNs in training data: 22911


In [60]:


import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Create synthetic data
np.random.seed(42)
num_samples = 10000
num_features = 286

# Generate random data for train_x (normally distributed)
train_x = np.random.randn(num_samples, num_features)

# Generate weights for each feature to introduce correlation between train_x and train_y
weights = np.random.randn(num_features)

# Generate train_y as a linear combination of train_x with some random noise
train_y = np.dot(train_x, weights) + np.random.randn(num_samples) * 0.5  # Adding noise

# Introduce NaNs into the dataset
train_x[::10] = np.nan  # Every 10th row will have NaN values

# Convert to pandas DataFrames for consistency with the original code
train_x = pd.DataFrame(train_x, columns=[f'feature_{i}' for i in range(num_features)])
train_y = pd.DataFrame(train_y, columns=['target'])

# Convert to PyTorch tensors
X_tensor = torch.tensor(train_x.values, dtype=torch.float32)
y_tensor = torch.tensor(train_y.values, dtype=torch.float32).squeeze()

# Replace NaNs with 0 for the forward passx
X_tensor = torch.nan_to_num(X_tensor, nan=0.0)

# Define the model
class CustomRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(CustomRegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(p=0.3)
       
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize model, optimizer, and loss function
model = CustomRegressionModel(input_dim=num_features)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    for i in range(0, X_tensor.size(0), batch_size):
        batch_X = X_tensor[i:i+batch_size]
        batch_y = y_tensor[i:i+batch_size]
       
        # Forward pass
        predictions = model(batch_X)
        loss = criterion(predictions.squeeze(), batch_y)
       
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
   
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 16.7323
Epoch [20/100], Loss: 11.7542
Epoch [30/100], Loss: 4.3444
Epoch [40/100], Loss: 8.4156
Epoch [50/100], Loss: 6.1261
Epoch [60/100], Loss: 10.6054
Epoch [70/100], Loss: 3.2802
Epoch [80/100], Loss: 3.2385
Epoch [90/100], Loss: 3.4161
Epoch [100/100], Loss: 6.7244


In [5]:
import h2o
from h2o.estimators import H2ODeepLearningEstimator, H2OAutoML
import numpy as np
import pandas as pd

# Initialize H2O cluster
h2o.init()

# Create synthetic data
np.random.seed(42)
num_samples = 10000
num_features = 286

# Generate random data for train_x (normally distributed)
train_x = np.random.randn(num_samples, num_features)

# Generate weights for each feature to introduce correlation between train_x and train_y
weights = np.random.randn(num_features)

# Generate train_y as a linear combination of train_x with some random noise
train_y = np.dot(train_x, weights) + np.random.randn(num_samples) * 0.5  # Adding noise

# Introduce NaNs into the dataset
train_x[::10] = np.nan  # Every 10th row will have NaN values

# Convert to pandas DataFrames for consistency
train_x = pd.DataFrame(train_x, columns=[f'feature_{i}' for i in range(num_features)])
train_y = pd.DataFrame(train_y, columns=['target'])

# Convert the pandas DataFrame to an H2OFrame
train_x_h2o = h2o.H2OFrame(train_x)
train_y_h2o = h2o.H2OFrame(train_y)

# Combine train_x and train_y for H2O training
train_h2o = train_x_h2o.cbind(train_y_h2o)

# Define features and target columns
features = train_x_h2o.columns
target = 'target'

# Initialize and train H2O Deep Learning model
model = H2ODeepLearningEstimator(
    hidden=[256, 128, 64],
    epochs=100,
    activation="RectifierWithDropout",
    input_dropout_ratio=0.2,
    hidden_dropout_ratios=[0.3, 0.3, 0.3],
    l1=1e-5,
    l2=1e-5
)

# Train the model
model.train(x=features, y=target, training_frame=train_h2o)

# Print the model performance
performance = model.model_performance()
print(performance)

# Alternatively, you could use AutoML to search for the best model
# aml = H2OAutoML(max_runtime_secs=600, project_name="regression")
# aml.train(x=features, y=target, training_frame=train_h2o)

# performance = aml.leader.model_performance()
# print(performance)

ModuleNotFoundError: No module named 'h2o'

In [2]:
import h2o
from h2o.estimators import H2ODeepLearningEstimator#, H2OAutoML
import numpy as np
import pandas as pd

# Initialize H2O cluster
h2o.init()

# Create synthetic data
np.random.seed(42)
num_samples = 10000
num_features = 286

# Generate random data for train_x (normally distributed)
train_x = np.random.randn(num_samples, num_features)

# Generate weights for each feature to introduce correlation between train_x and train_y
weights = np.random.randn(num_features)

# Generate train_y as a linear combination of train_x with some random noise
train_y = np.dot(train_x, weights) + np.random.randn(num_samples) * 0.5  # Adding noise

# Introduce NaNs into the dataset
train_x[::10] = np.nan  # Every 10th row will have NaN values

# Convert to pandas DataFrames for consistency
train_x = pd.DataFrame(train_x, columns=[f'feature_{i}' for i in range(num_features)])
train_y = pd.DataFrame(train_y, columns=['target'])

# Convert the pandas DataFrame to an H2OFrame
train_x_h2o = h2o.H2OFrame(train_x)
train_y_h2o = h2o.H2OFrame(train_y)

# Combine train_x and train_y for H2O training
train_h2o = train_x_h2o.cbind(train_y_h2o)

# Define features and target columns
features = train_x_h2o.columns
target = 'target'

# Initialize and train H2O Deep Learning model
model = H2ODeepLearningEstimator(
    hidden=[256, 128, 64],
    epochs=100,
    activation="RectifierWithDropout",
    input_dropout_ratio=0.2,
    hidden_dropout_ratios=[0.3, 0.3, 0.3],
    l1=1e-5,
    l2=1e-5
)

# Train the model
model.train(x=features, y=target, training_frame=train_h2o)

# Print the model performance
performance = model.model_performance()
print(performance)



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,9 hours 23 mins
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,25 days
H2O_cluster_name:,mattsalomon
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.997 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%
ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 55.29712009566408
RMSE: 7.43620333877874
MAE: 5.450701462911353
RMSLE: NaN
Mean Residual Deviance: 55.29712009566408


In [3]:
test_x = np.random.randn(num_samples, num_features)
test_y = np.dot(test_x, weights) + np.random.randn(num_samples) * 0.5  # Adding noise
test_x[::10] = np.nan
test_x_h2o = h2o.H2OFrame(test_x, column_names=[f'feature_{i}' for i in range(num_features)])
test_y_h2o = h2o.H2OFrame(test_y,column_names=['target'])

# Combine train_x and train_y for H2O training
test_h2o = test_x_h2o.cbind(test_y_h2o)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
train_x.shape, test_x.shape

((10000, 286), (10000, 286))

In [5]:
# Make predictions
train_pred = model.predict(train_h2o)
test_pred = model.predict(test_h2o)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [6]:
pd.DataFrame(data={'y_pred_train':train_pred.as_data_frame().values.flatten(), 'y_true_train':train_y.values.flatten()})

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,y_pred_train,y_true_train
0,-2.143833,-15.076742
1,3.036379,8.620448
2,19.299648,28.172118
3,26.547597,34.670782
4,-3.279449,-4.697201
...,...,...
9995,-15.412716,-20.868444
9996,-9.777033,-13.297913
9997,-27.374835,-33.682206
9998,5.994463,12.164308


In [7]:
# # Make predictions
# test_h2o = h2o.H2OFrame(test_x)
# predictions = model.predict(test_h2o)

# # View the predictions
# print(predictions.head())

In [8]:
from h2o.automl import H2OAutoML
# Alternatively, you could use AutoML to search for the best model
aml = H2OAutoML(max_runtime_secs=600, project_name="regression")
aml.train(x=features, y=target, training_frame=train_h2o)

performance = aml.leader.model_performance()
print(performance)

AutoML progress: |
06:12:21.856: New models will be added to existing leaderboard regression@@target (leaderboard frame=null) with already 22 models.

████████████████████████████████████████████████████
06:20:35.12: StackedEnsemble_BestOfFamily_4_AutoML_2_20240924_61221 [StackedEnsemble best_of_family_1 (built with AUTO metalearner, using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.

████████
06:21:54.936: StackedEnsemble_BestOfFamily_5_AutoML_2_20240924_61221 [StackedEnsemble best_of_family_2 (built with AUTO metalearner, using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
06:21:54.959: S

In [9]:
aml.leaderboard

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GLM_2_AutoML_2_20240924_61221,5.29464,28.0332,1.71254,nan,28.0332
GLM_1_AutoML_1_20240923_211230,5.29464,28.0332,1.71254,nan,28.0332
StackedEnsemble_AllModels_2_AutoML_1_20240923_211230,5.29509,28.038,1.71367,nan,28.038
StackedEnsemble_BestOfFamily_1_AutoML_1_20240923_211230,5.29512,28.0383,1.71415,nan,28.0383
StackedEnsemble_BestOfFamily_2_AutoML_1_20240923_211230,5.29513,28.0384,1.71412,nan,28.0384
StackedEnsemble_AllModels_1_AutoML_1_20240923_211230,5.29523,28.0395,1.71372,nan,28.0395
StackedEnsemble_AllModels_3_AutoML_1_20240923_211230,5.29554,28.0428,1.71425,nan,28.0428
StackedEnsemble_BestOfFamily_3_AutoML_1_20240923_211230,5.29639,28.0518,1.71507,nan,28.0518
DeepLearning_1_AutoML_1_20240923_211230,5.47443,29.9694,2.44844,nan,29.9694
DeepLearning_2_AutoML_2_20240924_61221,5.61972,31.5813,2.68302,nan,31.5813


In [10]:
perf = aml.leader.model_performance(test_h2o)

In [11]:
perf.rmse(), perf.mae(), perf.r2()

(5.584443849267315, 1.7659740471731105, 0.8880637559540919)

In [12]:
train_pred = aml.predict(train_h2o)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [13]:
pd.DataFrame(data={'y_pred_train':train_pred.as_data_frame().values.flatten(), 'y_true_train':train_y.values.flatten()}).describe()

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,y_pred_train,y_true_train
count,10000.000000,10000.000000
mean,0.068778,0.068778
std,15.727700,16.599437
min,-63.873488,-64.967615
25%,-9.638754,-11.136724
50%,0.068778,0.373693
75%,9.699891,11.108575
max,66.888468,67.440638


In [14]:
test_pred = aml.predict(test_h2o)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [17]:
pd.DataFrame(data={'y_pred_test':test_pred.as_data_frame().values.flatten(), 'y_true_test':test_y}).describe()

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,y_pred_test,y_true_test
count,10000.000000,10000.000000
mean,-0.140927,-0.098234
std,15.734737,16.692304
min,-58.230433,-58.609892
25%,-10.167124,-11.406969
50%,0.068778,-0.272202
75%,9.628301,11.239609
max,65.862511,64.911269


In [18]:
print(aml.leader.summary())

GLM Model: summary
    family    link      regularization               lambda_search                                                                  number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  --------  ---------------------------  -----------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------------------------------------
    gaussian  identity  Ridge ( lambda = 3.046E-4 )  nlambda = 30, lambda.max = 304.64, lambda.min = 3.046E-4, lambda.1se = 0.2401  286                           286                            30                      AutoML_2_20240924_61221_training_py_1_sid_a662


In [24]:
aml.leaderboard.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GLM_2_AutoML_2_20240924_61221,5.29464,28.0332,1.71254,nan,28.0332
GLM_1_AutoML_1_20240923_211230,5.29464,28.0332,1.71254,nan,28.0332
StackedEnsemble_AllModels_2_AutoML_1_20240923_211230,5.29509,28.038,1.71367,nan,28.038
StackedEnsemble_BestOfFamily_1_AutoML_1_20240923_211230,5.29512,28.0383,1.71415,nan,28.0383
StackedEnsemble_BestOfFamily_2_AutoML_1_20240923_211230,5.29513,28.0384,1.71412,nan,28.0384
StackedEnsemble_AllModels_1_AutoML_1_20240923_211230,5.29523,28.0395,1.71372,nan,28.0395
StackedEnsemble_AllModels_3_AutoML_1_20240923_211230,5.29554,28.0428,1.71425,nan,28.0428
StackedEnsemble_BestOfFamily_3_AutoML_1_20240923_211230,5.29639,28.0518,1.71507,nan,28.0518
DeepLearning_1_AutoML_1_20240923_211230,5.47443,29.9694,2.44844,nan,29.9694
DeepLearning_2_AutoML_2_20240924_61221,5.61972,31.5813,2.68302,nan,31.5813


In [31]:
aml.leaderboard.as_data_frame().iloc[8]

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


model_id                  DeepLearning_1_AutoML_1_20240923_211230
rmse                                                     5.474434
mse                                                     29.969427
mae                                                      2.448435
rmsle                                                         NaN
mean_residual_deviance                                  29.969427
Name: 8, dtype: object

In [39]:
# Access a specific model by name
model_id = aml.leaderboard.as_data_frame().iloc[8, 0]  # Extract the model ID as a string
print(model_id)
specific_model = h2o.get_model(model_id)

# Get the neural network structure
print(specific_model.summary())

# Get the loss function
print(specific_model.params['loss'])

DeepLearning_1_AutoML_1_20240923_211230
Status of Neuron Layers: predicting target, regression, gaussian distribution, Quadratic loss, 3,101 weights/biases, 77.1 KB, 104,018 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate               rate_rms                momentum    mean_weight             weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  ----------------------  ----------------------  ----------  ----------------------  -------------------  --------------------  -----------------------
    1        286      Input      0.0
    2        10       Rectifier  0.0        0.0   0.0   0.025230179532672037    0.009651198983192444    0.0         0.00023786274331370066  0.0954655110836029   0.7788656104215739    0.5845489501953125
    3        10       Rectifier  0.0        0.0   0.0   0.008876945667725521    0.008416924625635147    0.0         -0.004443328110501171   0.2827370166

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [63]:
# Access a specific model by name
model_id = aml.leaderboard.as_data_frame().iloc[8, 0]  # Get the model ID
specific_model = h2o.get_model(model_id)
model = specific_model
# Get model summary
print(specific_model.summary())  # This will show details about the layers, units, activation functions, etc.

Status of Neuron Layers: predicting target, regression, gaussian distribution, Quadratic loss, 3,101 weights/biases, 77.1 KB, 104,018 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate               rate_rms                momentum    mean_weight             weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  ----------------------  ----------------------  ----------  ----------------------  -------------------  --------------------  -----------------------
    1        286      Input      0.0
    2        10       Rectifier  0.0        0.0   0.0   0.025230179532672037    0.009651198983192444    0.0         0.00023786274331370066  0.0954655110836029   0.7788656104215739    0.5845489501953125
    3        10       Rectifier  0.0        0.0   0.0   0.008876945667725521    0.008416924625635147    0.0         -0.004443328110501171   0.28273701667785645  0.9809969561491746    0.0655244

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [76]:
import torch
import torch.nn as nn

# Define a PyTorch model with the same architecture as the H2O model
class ConvertedH2OModel(nn.Module):
    def __init__(self, input_dim):
        super(ConvertedH2OModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(p=0.3)
       
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize the PyTorch model
input_dim = num_features  # Assuming num_features is defined
torch_model = ConvertedH2OModel(input_dim)

# Check if the model has weights and biases
if specific_model._model_json["output"]["weights"] is not None and specific_model._model_json["output"]["biases"] is not None:
    # Extract weights and biases from the H2O model
    h2o_weights = [specific_model.weights(i) for i in range(len(specific_model.params['hidden']))]
    h2o_biases = [specific_model.biases(i) for i in range(len(specific_model.params['hidden']))]

    # Assign the extracted weights and biases to the PyTorch model
    with torch.no_grad():
        torch_model.fc1.weight = nn.Parameter(torch.tensor(h2o_weights[0].T, dtype=torch.float32))
        torch_model.fc1.bias = nn.Parameter(torch.tensor(h2o_biases[0], dtype=torch.float32))
        torch_model.fc2.weight = nn.Parameter(torch.tensor(h2o_weights[1].T, dtype=torch.float32))
        torch_model.fc2.bias = nn.Parameter(torch.tensor(h2o_biases[1], dtype=torch.float32))
        torch_model.fc3.weight = nn.Parameter(torch.tensor(h2o_weights[2].T, dtype=torch.float32))
        torch_model.fc3.bias = nn.Parameter(torch.tensor(h2o_biases[2], dtype=torch.float32))
        torch_model.fc4.weight = nn.Parameter(torch.tensor(h2o_weights[3].T, dtype=torch.float32))
        torch_model.fc4.bias = nn.Parameter(torch.tensor(h2o_biases[3], dtype=torch.float32))

    # Now the torch_model is initialized with the weights and biases from the H2O model
else:
    print("The model does not have weights and biases available for extraction.")


The model does not have weights and biases available for extraction.


In [75]:
 model._model_json["output"]["weights"]

In [56]:
import torch
import torch.nn as nn

class ConvertedH2OModel(nn.Module):
    def __init__(self, input_dim):
        super(ConvertedH2OModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(p=0.3)  # Adjust dropout based on your architecture

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize the PyTorch model
input_dim = 286  # Adjust based on your dataset
torch_model = ConvertedH2OModel(input_dim)

In [61]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(torch_model.parameters(), lr=0.001)

# Example training loop (assuming X_train and y_train are your training data)
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = torch_model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 275.5254
Epoch [20/100], Loss: 275.5214
Epoch [30/100], Loss: 275.5193
Epoch [40/100], Loss: 275.5181
Epoch [50/100], Loss: 275.5172
Epoch [60/100], Loss: 275.5167
Epoch [70/100], Loss: 275.5163
Epoch [80/100], Loss: 275.5161
Epoch [90/100], Loss: 275.5158
Epoch [100/100], Loss: 275.5157


In [48]:
model = h2o.get_model(model_id)
if hasattr(model, 'metalearner'):
    metalearner_params = model.metalearner().actual_params
else:
    metalearner_params = None
se_params = model.actual_params


In [19]:
print(aml.leader.algo)  # To check the type of algorithm (e.g., "GBM", "DeepLearning", etc.)# For deep learning models


glm


In [23]:
# layer_weights = aml.leader.coef(layer_number)
# layer_biases = aml.leader.biases(layer_number)

In [21]:
aml.leader.coef()

{'Intercept': -0.05383154654340083,
 'feature_0': 0.5272049781557735,
 'feature_1': 0.21592772001764768,
 'feature_2': 0.1540862842393204,
 'feature_3': 0.07236426622464344,
 'feature_4': -0.17708315292505578,
 'feature_5': -0.5324016895714361,
 'feature_6': -1.176444533475652,
 'feature_7': 0.899782570976865,
 'feature_8': 1.0532240069632302,
 'feature_9': 1.173530299599555,
 'feature_10': 1.8405155207005701,
 'feature_11': -0.8727583211265966,
 'feature_12': -1.8763701813875338,
 'feature_13': 0.2540544724877997,
 'feature_14': -1.66344180080521,
 'feature_15': 0.4331708455484692,
 'feature_16': -0.5358056313137141,
 'feature_17': -1.8954152669536164,
 'feature_18': -0.08352995719280579,
 'feature_19': 0.36538576313052973,
 'feature_20': 1.3202206727795143,
 'feature_21': 0.5991797451939189,
 'feature_22': 0.7828946605091546,
 'feature_23': -1.4058715506475654,
 'feature_24': -0.15409528486836158,
 'feature_25': -0.938312481036799,
 'feature_26': -1.5940512677919938,
 'feature_27': -

In [22]:
aml.leader.b

KeyError: 'biases'